In [30]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import nltk
import re
import string
import pymorphy2

def pymorphy2_311_hotfix():
    from inspect import getfullargspec
    from pymorphy2.units.base import BaseAnalyzerUnit

    def _get_param_names_311(klass):
        if klass.__init__ is object.__init__:
            return []
        args = getfullargspec(klass.__init__).args
        return sorted(args[1:])

    setattr(BaseAnalyzerUnit, '_get_param_names', _get_param_names_311)

### Задание

Выберите любую новость с любого новостного портала

• Проведите парсинг текста новости

• Очистите текст от стоп-слов

• Оцените тональность новости с помощью библиотеки dostoevsky

• В случае проблем с установкой dostoevsky проведите лемматизацию текста и выведите 5 самых частых слов в новости

### Загрузка новости

In [35]:
url = "https://www.ixbt.com/news/2024/11/26/kalifornija-protiv-trampa-i-maska-shtat-sobiraetsja-davat-skidki-na-vse-jelektromobili-krome-tesla.html"
page = requests.get(url)
print(page.status_code)
soup = ""
if page.status_code == 200:
    soup = BeautifulSoup(page.text, "html.parser")

soup

200


 <!DOCTYPE html>

<html lang="ru">
<head>
<link href="https://www.ixbt.com/news/2024/11/26/kalifornija-protiv-trampa-i-maska-shtat-sobiraetsja-davat-skidki-na-vse-jelektromobili-krome-tesla.html" rel="canonical"/>
<link href="https://www.ixbt.com/news/2024/11/26/kalifornija-protiv-trampa-i-maska-shtat-sobiraetsja-davat-skidki-na-vse-jelektromobili-krome-tesla.amp.html" rel="amphtml"/>
<meta charset="utf-8"/>
<meta content="initial-scale=1" name="viewport"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="false" property="relap:article"/>
<title>Губернатор Калифорнии против Трампа и Маска. Штат собирается давать скидки на все электромобили, кроме Tesla</title>
<meta content="Tesla, авто" name="keywords">
<meta content="Губернатор Калифорнии Гэвин Ньюсом объявил, что, если Трамп решит отменить федеральную политику налоговых льгот для электромобилей после вступления в должность, Калифорния предоставит отдельную налоговую льготу покупателям электромобилей." name="desc

In [7]:
news_header = soup.find(class_ = 'b-article__header').get_text()
news_body = soup.find(class_ = 'b-article__content').get_text()
text = news_header + '\n' + news_body
text

'\nГубернатор Калифорнии против Трампа и Маска. Штат собирается давать скидки на все электромобили, кроме Tesla\n\nСуществующую скидку 7500 долларов при Трампе должны отменить \n\n\nГубернатор Калифорнии Гэвин Ньюсом объявил, что, если Трамп решит отменить федеральную политику налоговых льгот для электромобилей после вступления в должность, Калифорния предоставит отдельную налоговую льготу покупателям электромобилей.\nНо он также упомянул, что нынешнее предложение не будет распространяться на Tesla. Конкретные детали необходимо постепенно согласовывать с государственным собранием, поэтому могут быть корректировки. «Речь идет о создании рыночных условий, которые позволят большему количеству автопроизводителей укорениться», — заявили в администрации губернатора.\nДо этого Соединенные Штаты предоставляли налоговые льготы в размере до 7500 долларов покупателям соответствующих электромобилей, но Трамп, как ожидается, планирует отменить эту политику.\n\n\nИзображение Midjourney\n\n\nНьюсом з

In [25]:
words_list = text.lower().translate(str.maketrans('', '', string.punctuation)).translate(str.maketrans('', '', string.digits)).split()

### Удаление стоп-слов

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Игорь\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
from nltk.corpus import stopwords
stopwords_ru = stopwords.words("russian")
words_list2 = [word for word in words_list if word not in stopwords_ru]
print(len(words_list), len(words_list2))

181 175


### Лемматизация текста

С установкой dostoevsky возникли проблемы. Терпение, выделенное на борьбу с ними иссякло.

In [36]:
pymorphy2_311_hotfix()
morph = pymorphy2.MorphAnalyzer()
lemmatized_words = [morph.parse(word)[0].normal_form for word in words_list2]
data = pd.DataFrame(lemmatized_words, columns=["words"])
words_count = data['words'].value_counts().reset_index()
words_count.columns = ['word', 'count']
words_count.head(5)

,word,count
0,калифорния,5
1,штат,5
2,электромобиль,4
3,трамп,4
4,автомобиль,4
